In [8]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import RobustScaler

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [3]:
import os
os.getcwd()

'/home/kacper/Repos/wb2021/proj2/DNN'

In [12]:
train_exp = pd.read_csv("../data/splitted/expired_train.csv").iloc[:, 1:]
test_exp = pd.read_csv("../data/splitted/expired_test.csv").iloc[:, 1:]

X_train_exp = np.array(train_exp.iloc[:, :-1])
y_train_exp = np.array(train_exp.iloc[:, -1])
X_test_exp = np.array(test_exp.iloc[:, :-1])
y_test_exp = np.array(test_exp.iloc[:, -1])

In [10]:
def make_article_layer(neurons, **kwargs):
    return Dense(neurons, activation="relu", kernel_initializer="he_normal", kernel_regularizer="l2", **kwargs)

def make_article_model(loss="mean_squared_error", input_shape=(5,)):
    # The DNN model used five hidden layers with 6, 8, 16, 8, 4 neurons respectively.
    model = Sequential()
    model.add(make_article_layer(6, input_shape=input_shape))
    model.add(make_article_layer(8))
    model.add(make_article_layer(16))
    model.add(make_article_layer(8))
    model.add(make_article_layer(4))
    # [...] the sigmoid activation function for the output layer, and the “he_normal” normalization scheme were applied. 
    model.add(Dense(1, activation="sigmoid", kernel_initializer="he_normal", kernel_regularizer="l2"))
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

def make_modified(input_shape=(5,)):
    return make_article_model(input_shape=input_shape, loss="binary_crossentropy")

def make_basic(input_shape=(5,)):
    wider = Sequential()
    wider.add(Dense(32, activation="relu", input_shape=input_shape))
    wider.add(Dense(32, activation="relu"))
    wider.add(Dense(1, activation="sigmoid"))
    wider.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return wider

In [5]:
def calc_roc_keras(model, X_test, y_test): 
    y_pred_keras = model.predict(X_test).ravel()
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)
    return fpr_keras, tpr_keras

In [9]:
def plot_rocs(rates: [(np.array, np.array)]):
    plt.plot(fpr, tpr)
    plt.plot([0,1], [0,1], "r--")
    plt.xlabel("FPR")
    plt.ylabel("TPR")

In [15]:
def make_and_fit_keras(constructor, X_train, y_train, icu=True):
    if icu:
        input_shape=(5,)
    else:
        input_shape=(6,)
    model = constructor(input_shape=input_shape)
    model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=0)
    return model

In [16]:
exp_article_model = make_and_fit_keras(make_article_model, X_train_exp, y_train_exp, icu=False)

ValueError: in user code:

    /home/kacper/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/kacper/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/kacper/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/kacper/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/kacper/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/kacper/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/